## 使用新方法做弯曲检测的版本
避免压线筒污染的影响， 但是效果不好

In [5]:
import sys
sys.path.append('/Users/zhouxinyu/侧边栏文件夹/Code/VSCode/Jupyter/Project/ZF/测试[v2]-22-07-28')
from data_verification import imgs as zf_test_imgs

In [16]:
import cv2
import numpy as np
import math

# 裁剪完成后进行分割
def image_cut(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    img = img[:, :-10, :]
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    lower_blue = np.array([100, 43, 46])
    upper_blue = np.array([124, 255, 255])
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    img_morph = mask.copy()
    core = (3, 3)
    cv2.erode(img_morph, core, img_morph, iterations=4)
    cv2.dilate(img_morph, core, img_morph, iterations=1)
    img_morph = cv2.medianBlur(img_morph, 15)
    res = cv2.findContours(img_morph, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    cnts = res[0]

    cnts_sort = sorted(cnts, key=cv2.contourArea, reverse=True)
    white_img = np.zeros_like(img)
    white_img[:, :, :] = 0
    cv2.drawContours(white_img, cnts_sort, 0, (255, 255, 255), -1)
    black_img = np.zeros_like(img)
    black_img[:, :, :] = 255
    black_img_copy = cv2.cvtColor(black_img, cv2.COLOR_BGR2GRAY)
    cv2.drawContours(black_img_copy, cnts_sort, 0, (0, 0, 0), -1)

    res = cv2.findContours(black_img_copy, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    img_copy = img.copy()
    cv2.drawContours(img_copy, res[0][0], -1, (0, 255, 0), 10)

    x, y, w, h = cv2.boundingRect(res[0][0])
    pure_img = img.copy()
    # 获得方框
    cv2.rectangle(pure_img, (x, y), (x + w, y + h), (0, 255, 0), 2)

    ## 灰度图
    gray_pure_img = cv2.cvtColor(pure_img, cv2.COLOR_BGR2GRAY)

    # 裁剪
    cuted_image = img[y:y + h, x:x + w]

    bounding = res[0][0] # 最大的contour
    
    #让contour的所有点的坐标移动，以配合裁剪出来的新图片
    def bounding_move(bounding_point):
        bounding_point[0, 0] -= x #裁剪范围的左上角的x坐标
        bounding_point[0, 1] -= y
        return bounding_point

    bounding = np.array(list(map(bounding_move, bounding)))
    return cuted_image, bounding

def fill_black(image, bounding):
    stencil = np.zeros(image.shape).astype(image.dtype)
    color = (255, 255, 255)
    cv2.fillPoly(stencil, [bounding], color)
    result = cv2.bitwise_and(image, stencil)
    return result

def vector_angle(line1: list, line2: list):
    """
    计算两条直线的夹角，
    :param line1: 直线一[x1, y1, x2, y2]
    :param line2: 直线二 [x1, y1, x2, y2]
    :return:
    """

    vector1 = np.array([line1[2] - line1[0], line1[3] - line1[1]])  # (x2-x1,y2-y1)
    vector2 = np.array([line2[2] - line2[0], line2[3] - line2[1]])
    return np.arccos(
        vector1 @ vector2 / (np.linalg.norm(vector1) * np.linalg.norm(vector2)))

def find_cut_point(image):
    
    height, width, color = image.shape
    image_HSV = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    step = 5
    
    x_list = []
    num_of_noBlack_points = []
        
    noBlack_sum = 0
    # 从右往左，统计每一列非黑色点的个数，也就是线缆的宽度，两列之间的间隔为step
    # 结果储存在num_of_noBlack_points里，因为是从右往左检查列的，
    # 所以num_of_noBlack_points的第一个元素为最右侧一列像素点的个数
    num_of_noBlack_start_x =  9 * width // 10 - 1 #在统计非黑色像素点时，不统计压线筒最右侧的圆头部分
    for x in range(num_of_noBlack_start_x, 0, -step):
        cnt = 0
        for y in range(0, height):
            if(image_HSV[y][x][2]!=0):
                cnt += 1
        num_of_noBlack_points.append(cnt)
        x_list.append(x)
        noBlack_sum += cnt
    noBlack_average = noBlack_sum // len(num_of_noBlack_points)
    # print("average:",noBlack_average)
    
    # 将num_of_noBlack_points划分为多个check_range，
    # 检查每个check_range里
    right_start = len(num_of_noBlack_points) // 100
    left_end = len(num_of_noBlack_points)
    check_range = len(num_of_noBlack_points) // 80
    
    cnt = 0
    bottom = height # 某个check_range内的最小值，所以一开始赋值为一个比较大的数字
    top = 0 # 某个check_range内的最大值，所以一开始赋值为一个比较小的数字
    delta_list = []
    delta_list_x = []
    
    x_Position = -1
    
    # 从左到右遍历num_of_noBlack_points，因为其中的值记录的是从右到左的线缆宽度，
    # 所以其实是从右向左遍历线缆宽度
    range_top_x = -1
    range_bottom_x = -1
    range_delta_flag = 0
    threshold = noBlack_average // 6
    for i in range(right_start, left_end, 1):
        if(cnt==0):
            range_delta = top - bottom # 如果检查完check_range内的所有点，就计算一下check_range内线缆最大和最小宽度的差
            if (range_bottom_x - range_top_x) <= 0: #在num_of_noBlack_points曲线图中，range内最低点应该在最高点右侧
                range_delta = -1 * range_delta
            delta_list.append(range_delta)
            cnt = check_range # 重新设置“倒计时”，如果倒数结束，说明已经检查了check_range个x
            bottom = height
            top = 0
            range_top_x = -1
            range_bottom_x = -1
            if(range_delta_flag != 0 and range_delta > threshold): # 如果找到了改变很大的点，break
                x_Position = x_list[i] + math.floor(check_range * step * 1.2)
                break
            range_delta_flag = 1
        cnt = cnt - 1
        if(num_of_noBlack_points[i] < bottom):
            bottom = num_of_noBlack_points[i]
            range_bottom_x = x_list[i]
        if(num_of_noBlack_points[i] > top):
            top = num_of_noBlack_points[i]
            range_top_x = x_list[i]    
    return x_Position

def get_two_lines(fill_black_right, orig_fill_b, x_Position):
    height, width, color = fill_black_right.shape
    fill_black_right_HSV = cv2.cvtColor(fill_black_right, cv2.COLOR_BGR2HSV)
    step = 5 #original step = 5

    # left line ---------------------------------------
    left_line_points_x = []
    left_line_points_y = []
    left_x_start = width//100 # 1/100
    left_x_end = 3 * width//40 # 3/40
    for x in range(left_x_start,left_x_end,step):
        left_line_points_x.append(x)
        temp_top = -1
        temp_bottom = -1
        for y in range(0, height):
            if fill_black_right_HSV[y][x][2] != 0:
                temp_top = y
                break
        for y in range(height - 1, 0, -1):
            if fill_black_right_HSV[y][x][2] != 0:
                temp_bottom = y
                break
        left_line_points_y.append( (temp_top + temp_bottom) // 2 )
    
    #right line ------------------------------------------
    right_line_points_x = []
    right_line_points_y = []
    right_x_start = width - width//5 # 16/20
    right_x_end = width - width//20 # 19/20
    for x in range(right_x_start,right_x_end,step):
        right_line_points_x.append(x)
        temp_top = -1
        temp_bottom = -1
        for y in range(0, height):
            if fill_black_right_HSV[y][x][2] != 0:
                temp_top = y
                break
        for y in range(height - 1, 0, -1):
            if fill_black_right_HSV[y][x][2] != 0:
                temp_bottom = y
                break
        right_line_points_y.append( (temp_top + temp_bottom) // 2 )
    
    #fitting ------------------------------------------
    left_line = np.polyfit(left_line_points_x, left_line_points_y, 1)
    right_line = np.polyfit(right_line_points_x,right_line_points_y,1)
    
    left_point1_x = 0
    left_point1_y = left_line[1]
    left_point2_x = 1
    left_point2_y = left_line[0] + left_line[1]
    
    right_point1_x = 0
    right_point1_y = right_line[1]
    right_point2_x = 1
    right_point2_y = right_line[0] + right_line[1]
    
    return [[left_point1_x,left_point1_y],[left_point2_x,left_point2_y],[right_point1_x,right_point1_y],[right_point2_x,right_point2_y]]

def if_bend(img_dir):
    
    cuted_image, bounding = image_cut(img_dir)
       
    fill_b = fill_black(cuted_image, bounding)
       
    x_Position = find_cut_point(fill_b)
       
    orig_fill_b = fill_b.copy()
    
    height, width, color = fill_b.shape
    margin = width // 10
    fill_black_right = fill_b[0:height,x_Position:width-margin]
    fourpoints = get_two_lines(fill_black_right,orig_fill_b,x_Position)
    
    line1 = [fourpoints[0][0],fourpoints[0][1], fourpoints[1][0],fourpoints[1][1]]
    line2 = [fourpoints[2][0],fourpoints[2][1], fourpoints[3][0],fourpoints[3][1]]
    a = vector_angle(line1,line2)
    pi = 3.1415926
    result = a/pi*180
    # print("result: ",result,"°")
    
    return result > 5, result

In [17]:
for i, filename in enumerate(zf_test_imgs.barrel_bending):
    print(f'{i}: {if_bend(filename)}')

0: (True, 6.9816158476135675)
1: (True, 9.961042660857158)
2: (True, 10.780603572015586)
3: (True, 21.838077708309925)
4: (True, 18.476059879790036)
5: (False, 3.180800212127204)
6: (False, 3.0643723784297787)
7: (True, 6.357695397849469)
8: (True, 8.789767098359745)
9: (True, 10.716833231911703)
10: (False, 4.3740779154760245)
11: (True, 11.888971588893984)
12: (True, 11.708435562942027)
13: (True, 13.881276757482409)
14: (True, 14.036875874849061)
15: (True, 14.596992150283043)
16: (True, 16.182541127249852)
17: (True, 13.393111340267886)
18: (True, 18.55224558223182)
19: (True, 15.388282676357875)
20: (True, 17.10285509487393)


In [18]:
for i, filename in enumerate(zf_test_imgs.barrel_pollution):
    print(f'{i}: {if_bend(filename)}')

0: (False, 2.011702988839124)
1: (False, 1.344121016058975)
2: (False, 0.2989760294262972)
3: (False, 1.2649290800543116)
4: (False, 1.2767973301836166)
5: (False, 1.2282928523474832)
6: (False, 2.477569726886696)
7: (False, 0.28792394991554754)
8: (False, 0.008262980419704155)
9: (False, 2.6708756465072225)
10: (False, 0.20710653916017255)
11: (False, 2.6065118035761667)
12: (False, 0.9152723472362808)
13: (True, 9.215552292013976)
14: (False, 1.8107177410955868)
15: (False, 2.180889214631131)
16: (False, 1.661081780382019)
17: (False, 1.2527053352500068)
18: (False, 1.1462257461664156)
19: (False, 1.406698570186466)
20: (False, 0.16360123474693278)
21: (False, 0.1349463225642468)
22: (False, 1.8270783812849758)
23: (False, 0.5758430589807781)
24: (False, 0.6821242762639033)
25: (False, 1.9823115157177107)
26: (False, 1.0656461868379716)
27: (False, 1.555320998052232)
28: (True, 6.963384481198692)
29: (False, 3.2840771037699175)
30: (False, 2.150513898328555)
31: (False, 1.68234660211

error: OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [19]:
for i, filename in enumerate(zf_test_imgs.line_damage):
    print(f'{i}: {if_bend(filename)}')

0: (False, 0.9461989094339693)
1: (False, 0.004704627784588643)
2: (False, 1.901799746543535)
3: (False, 3.5855101930888407)
4: (False, 1.7891942018688616)
5: (False, 1.708162931967972)


error: OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
